In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [5]:
import pickle

In [6]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment-1')

<Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='nyc-taxi-experiment-1', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df["duration"] = pd.to_datetime(df.lpep_dropoff_datetime) - pd.to_datetime(df.lpep_pickup_datetime)
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID","DOLocationID"]

    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe("./data/green_tripdata_2021-01.parquet")
df_validation = read_dataframe("./data/green_tripdata_2021-02.parquet")

In [9]:
len(df_train), len(df_validation)

(73908, 61921)

In [10]:
df_train["PU_DO"] = df_train["PULocationID"] + df_train["DOLocationID"]
df_validation["PU_DO"] = df_validation["PULocationID"] + df_validation["DOLocationID"]

In [11]:
categorical = ["PU_DO"] #"PULocationID","DOLocationID"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dict)

val_dict = df_validation[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dict)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_validation[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.787351952580485

In [26]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "Vijay")
    
    mlflow.log_param("train_data_path","./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation_data_path","./data/green_tripdata_2021-02.parquet")
    
    alpha = 0.01
    
    mlflow.log_param("alpha",alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse",rmse)
    
    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path='models_pickle')

In [18]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.342358343665872

In [19]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [14]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [15]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[11:22:04] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.46960                                                    
[1]	validation-rmse:8.16460                                                     
[2]	validation-rmse:7.18668                                                     
[3]	validation-rmse:6.87734                                                     
[4]	validation-rmse:6.76900                                                     
[5]	validation-rmse:6.72393                                                     
[6]	validation-rmse:6.70558                                                     
[7]	validation-rmse:6.69790                                                     
[8]	validation-rmse:6.69344                                                     
[9]	validation-rmse:6.68759                                                     
[10]	validation-rmse:6.68367                                           

KeyboardInterrupt: 

In [24]:
best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

mlflow.xgboost.autolog()

booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

2022/06/21 11:43:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '99a441bed98942b4894086e81a47001c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[11:43:07] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48424
[1]	validation-rmse:17.95632
[2]	validation-rmse:16.59111
[3]	validation-rmse:15.37410
[4]	validation-rmse:14.29018
[5]	validation-rmse:13.32803
[6]	validation-rmse:12.47574
[7]	validation-rmse:11.72161
[8]	validation-rmse:11.05920
[9]	validation-rmse:10.47573
[10]	validation-rmse:9.96569
[11]	validation-rmse:9.51821
[12]	validation-rmse:9.12657
[13]	validation-rmse:8.78542
[14]	validation-rmse:8.49042
[15]	validation-rmse:8.23359
[16]	validation-rmse:8.01257
[17]	validation-rmse:7.82059
[18]	validation-rmse:7.65364
[19]	validation-rmse:7.50842
[20]	validation-rmse:7.38327
[21]	validation-rmse:7.27530
[22]	validation-rmse:7.18023
[23]	validation-rmse:7.09773
[24]	validation-rmse:7.02688
[25]	validation-rmse:6.96485
[26]	validation-rmse:6.91103
[27]	validation-rmse:6.86367
[28]	validation-rmse:6.82252
[29]	validation-rmse:6.78611
[30]	valida

2022/06/21 11:45:02 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/06/21 11:45:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [16]:
mlflow.xgboost.autolog(disable=True)

In [17]:
with mlflow.start_run():
    
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
    
    with open('models/preprocessor.b','wb') as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact('models/preprocessor.b',artifact_path="preprocessor")
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.xgboost.log_model(booster, artifact_path = "models_mlflow")

[12:40:39] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48424
[1]	validation-rmse:17.95632
[2]	validation-rmse:16.59111
[3]	validation-rmse:15.37410
[4]	validation-rmse:14.29018
[5]	validation-rmse:13.32803
[6]	validation-rmse:12.47574
[7]	validation-rmse:11.72161
[8]	validation-rmse:11.05920
[9]	validation-rmse:10.47573
[10]	validation-rmse:9.96569
[11]	validation-rmse:9.51821
[12]	validation-rmse:9.12657
[13]	validation-rmse:8.78542
[14]	validation-rmse:8.49042
[15]	validation-rmse:8.23359
[16]	validation-rmse:8.01257
[17]	validation-rmse:7.82059
[18]	validation-rmse:7.65364
[19]	validation-rmse:7.50842
[20]	validation-rmse:7.38327
[21]	validation-rmse:7.27530
[22]	validation-rmse:7.18023
[23]	validation-rmse:7.09773
[24]	validation-rmse:7.02688
[25]	validation-rmse:6.96485
[26]	validation-rmse:6.91103
[27]	validation-rmse:6.86367
[28]	validation-rmse:6.82252
[29]	validation-rmse:6.78611
[30]	valida

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [18]:
logged_model = 'runs:/5954872f22d84a64b9aa4be96f14f1da/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

[12:42:45] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [19]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 5954872f22d84a64b9aa4be96f14f1da

In [20]:
xg_model = mlflow.xgboost.load_model(logged_model)

[12:43:21] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [21]:
xg_model

In [22]:
y_pred = xg_model.predict(valid)

In [23]:
y_pred[:10]

array([14.989478,  7.139387, 14.210059, 24.33779 ,  9.306994, 17.147194,
       11.555898,  8.745514,  8.940153, 20.389336], dtype=float32)